In [2]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import load
import os
import sys

In [3]:
#model

class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, num_layers, dropout):
        super(RegressionModel, self).__init__()

        layers = []
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.ReLU())

        for i in range(1, num_layers - 1):
            layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        
        layers.append(nn.Linear(hidden_sizes[num_layers - 2], output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [4]:
#model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_sizes = [512, 256, 128, 64]
model = RegressionModel(input_size=5, hidden_sizes=hidden_sizes, output_size=1, num_layers=5, dropout=0.1)
model

RegressionModel(
  (model): Sequential(
    (0): Linear(in_features=5, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.1, inplace=False)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.1, inplace=False)
    (11): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [10]:
#evaluate patient by patient

def evaluate(organ):

    #model loading for the organ
    model.load_state_dict(torch.load('models/'+ organ + '_model.pth'))
    model.to(device)

    #scaler for the organ
    data = load('training_arrays/merged_training_arrays/'+ organ + '_training_array.npy')

    X = data[:,:-1]

    # Find rows containing NaN values in X
    nan_rows_X = np.isnan(X).any(axis=1)

    # Drop rows with NaN values from X 
    X = X[~nan_rows_X]

    # fit scaler 
    min_max_scaler = MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    
    # Save the original standard output
    original_stdout = sys.stdout

    # Define the file name for saving the output
    output_filename ='validation_results/' + organ + '_validation_results.txt'

    # Open the file for writing
    with open(output_filename, 'w') as f:
        # Redirect the standard output to the file
        sys.stdout = f

        files = os.listdir('training_arrays/validation_cohort/' + organ)
        diffs = []
        
        for file in files:
            
            print(file)
            
            test_data = load('training_arrays/validation_cohort/' + organ + '/' + file)
            X_test = test_data[:,:-1]
            y_test = test_data[:,-1]
            
            # Find rows containing NaN values in X
            nan_rows_X_test = np.isnan(X_test).any(axis=1)

            # Drop rows with NaN values from X and y
            X_test = X_test[~nan_rows_X_test]
            y_test = y_test[~nan_rows_X_test]
            
            #scaling
            X_test = min_max_scaler.transform(X_test)

            #data to tensor
            X_test = torch.tensor(X_test, dtype=torch.float32).to(device)

            #evaluate
            with torch.no_grad():
                model.eval()
                predictions = model(X_test).cpu().numpy()

            diffs.append(abs(100*((np.mean(y_test) - np.mean(predictions))/np.mean(y_test))))

            print(f'Ground Truth dose (mGy): {np.mean(y_test):.1f}')
            print(f'Predicted dose (mGy): {np.mean(predictions):.1f}')
            print(f'Percent Error:  {100*((np.mean(y_test) - np.mean(predictions))/np.mean(y_test)):.1f}%')
            print('----------------------')
        print(f'Mean Percentage Error: {np.nanmean(diffs):.1f}%')
        print(f'Max Percentage Error: {np.nanmax(diffs):.1f}%')
        print(f'Min Percentage Error: {np.nanmin(diffs):.1f}%')
        print(f'Standard Deviation: {np.nanstd(diffs):.1f}')
        sys.stdout = original_stdout


In [12]:
organs_list = ['esophagus', 'aorta', 'pulmonary_artery', 'lung_lower_lobe_left', 'lung_lower_lobe_right',
                    'lung_upper_lobe_left', 'lung_upper_lobe_right', 'lung_middle_lobe_right', 
                    'heart_atrium_left', 'heart_atrium_right', 'heart_ventricle_left', 'heart_ventricle_right', 
                    'heart_myocardium', 'skin']

for organ in organs_list:
    evaluate(organ)

In [8]:
evaluate('pulmonary_artery')